In [1]:
import tensorflow as tf
import numpy as np
import efficientnet.keras as efn 
from efficientnet.keras import preprocess_input

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size_all    = 5
epochs_all        = 100
#imgs_train        = 375787
#imgs_val          = 19832
imgs_train        = 762253
imgs_val          = 40497
path_train        = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/'
path_val          = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\val/'

In [3]:
#https://github.com/yu4u/cutout-random-erasing
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=True):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        if np.random.rand() > p: return input_img
        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)
            if left + w <= img_w and top + h <= img_h: break
        if np.random.rand() >0.5: c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else: c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c
        return input_img
    return eraser


gen_t = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    preprocessing_function=get_random_eraser(),
)

gen_v = ImageDataGenerator(
)

In [4]:
base_model = efn.EfficientNetB3(weights=None)
base_model.layers.pop()
base_model.layers.pop()

x = Dense(3, activation='softmax')(base_model.layers[-1].output)

model = Model(inputs=base_model.input, outputs=[x])

#load weights for the second round
#model.load_weights('effnet_ep003-loss5.584-val_loss0.001.h5') 
model.load_weights(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_model\effnet_ep005-loss67.592-val_loss46.489.h5') 












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [ ]:
def categorical_focal_loss(y_true, y_pred):
    gamma=2.
    alpha=.25
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    cross_entropy = -y_true * K.log(y_pred)
    loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
    return K.mean(loss, axis=1)*10000.0 #to avoid 0.0000





for layer in model.layers: layer.trainable = True
    
model.compile(optimizer=Adadelta(0.2), loss=categorical_focal_loss, metrics=['accuracy'])
train_generator = gen_t.flow_from_directory(path_train, target_size=(300,300), batch_size=batch_size_all)
valid_generator = gen_v.flow_from_directory(path_val,   target_size=(300,300), batch_size=batch_size_all)

#checkpoint      = ModelCheckpoint(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_model\effnet_ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5', monitor='val_loss', save_weights_only=True, save_best_only=False,verbose=1)
checkpoint      = ModelCheckpoint(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_model\effnet_ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}-val_acc{val_acc:.3f}.h5', monitor='val_loss', save_weights_only=True, save_best_only=False,verbose=1)
#reduce_lr       = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, verbose=1, delta=0.03)
reduce_lr       = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3, verbose=1, delta=0.03)

model.fit_generator(train_generator,  steps_per_epoch=1000, epochs=epochs_all, 
                    validation_data=valid_generator, validation_steps=np.ceil(imgs_val//batch_size_all),
                    callbacks=[reduce_lr, checkpoint])

In [7]:
#with this functionality we show boxes with incorect classification. we check them and delete manually boxes
#with incorrect classification
# 誤分類した画像を表示します。

import os
import matplotlib.pyplot as plt
import cv2 as cv
import shutil

target_class = 2 #realize for 0, 1, 2
bad_imgs = 0
dir_imgs_name   = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/'+str(target_class)
move_path       = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\exclusion/'
#dir_imgs_name   = 'C:/Users/Hurtik/python-petr/signate_3rd_ai_edge_competiton/train_imgs_class/val/'+str(target_class)
list_of_imgs = [root+"/"+name
             for root, dirs, files in os.walk(dir_imgs_name)
             for name in files]    

    
for im in range (0, len(list_of_imgs), 1):
    imgcv = cv.cvtColor(cv.imread(list_of_imgs[im]),cv.COLOR_RGB2BGR)
    pred = np.argmax(model.predict(np.expand_dims(imgcv, 0))[0])
    if pred != target_class:
        print(str(im) + ":", list_of_imgs[im])
        shutil.move(list_of_imgs[im], move_path)
        #plt.imshow(imgcv)
        #plt.show()
        bad_imgs += 1
print('bad_imgs: ', bad_imgs)

39: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100030.jpg
124: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100118.jpg
181: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_10017.jpg
481: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100450.jpg
515: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100481.jpg
581: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100540.jpg
648: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100611.jpg
691: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100650.jpg
727: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100683.jpg
787: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100737.jpg
814: D:\SIGNATE\Signate_3rd_AI_edge_competition\classifier_images\train/2/2_100761.jpg
1101: D:\SIGNATE\Signate_3rd_AI_edge_competit

KeyboardInterrupt: 